# 1. Load the JSON data

In [3]:
import json

import pandas as pd

/Users/natashiblie/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [76]:
with open('../data/multicity_historical.json', 'r') as file:
    data = json.load(file)

data.keys()

dict_keys(['London', 'Singapore', 'Cairo', 'Buenos Aires', 'Mumbai'])

Let's make an empty list to hold dataframes for each city.

In [77]:
city_dfs = []

Since we have 5 cities, we can use the loop function to stucture it into dataframes.

In [78]:
for city, values in data.items():
    city_df = pd.DataFrame(values)
    city_df['city'] = city  # Add a column for the city name
    city_dfs.append(city_df)


Let's concatenate all the city dataframes into a single dataframe.

In [79]:
df = pd.concat(city_dfs, ignore_index=True)

Display the first few rows to check the structure.

I've used the display() function I searched on Copilot that displays the output in the same table format as W03 Lab NB02 notebooks. 

In [80]:
display(df.head())

,time,precipitation_sum,precipitation_hours,city
0,2023-01-01,4.0,12.0,London
1,2023-01-02,0.2,2.0,London
2,2023-01-03,3.2,14.0,London
3,2023-01-04,0.9,5.0,London
4,2023-01-05,0.1,1.0,London


Display the last few rows to check the structure.

In [81]:
display(df.tail())

,time,precipitation_sum,precipitation_hours,city
1820,2023-12-27,0.0,0.0,Mumbai
1821,2023-12-28,0.0,0.0,Mumbai
1822,2023-12-29,0.0,0.0,Mumbai
1823,2023-12-30,0.0,0.0,Mumbai
1824,2023-12-31,0.0,0.0,Mumbai


I also want to check that there should be a total of 1825 entries (365 days x 5 cities).

In [82]:
entries = len(df)
print(entries)

1825


# 2. Calculate Key Metrics for Raininess Analysis

Now that we have obtained the relevant data and sorted it into organised tables, we can calculate the following required metrics. 

We will be using groupby() function to obtain the data from the coreresponding columns required.

- Total Rainfall: Sum of precipitation_sum for each city.

- Number of Rainy Days: Days with precipitation_sum > 0

- Average Rain Intensity: Total Rainfall / Number of Rainy Days

- Average Rain Duration: Total precipitation_hours / Number of Rainy Days



## 2.1 Total Rainfall

In [83]:
# Group by 'city' and sum the 'precipitation' for each group
total_rainfall = df.groupby('city')['precipitation_sum'].sum().reset_index()

# First column is the city name, second column is the sum of precipitation
total_rainfall.columns = ['City', 'Total Rainfall']

# Display the result
display(total_rainfall)


,City,Total Rainfall
0,Buenos Aires,916.3
1,Cairo,28.3
2,London,780.7
3,Mumbai,2048.1
4,Singapore,2364.7


Above, we can see there's no column header for total_rainfall. I want to add the column header, and based on Copilot, I can use reset_index()

In [84]:
# Group by 'city' and sum the 'precipitation' for each group
total_rainfall = df.groupby('city')['precipitation_sum'].sum().reset_index()

# Rename the columns for clarity where first column is the city name, second column is the sum of precipitation
total_rainfall.columns = ['City', 'Total Rainfall']

# Display the result
display(total_rainfall)


,City,Total Rainfall
0,Buenos Aires,916.3
1,Cairo,28.3
2,London,780.7
3,Mumbai,2048.1
4,Singapore,2364.7


## 2.2 Number of Rainy Days

We introduce the size() function to count the number of occurences in each group. In this case, we want to count the number of days precipitation_sum is greater than 0.

In [85]:
# Filter the DataFrame to include only rows where precipitation_sum > 0
rainy_days_df = df[df['precipitation_sum'] > 0]

# Group by 'city' and count the number of days for each city
rainy_days = rainy_days_df.groupby('city').size().reset_index()

# Rename the columns for clarity
rainy_days.columns = ['City', 'Rainy Days']

# Display the result
display(rainy_days)

,City,Rainy Days
0,Buenos Aires,150
1,Cairo,29
2,London,228
3,Mumbai,157
4,Singapore,350


## 2.3 Average Rain Intensity

Now, we can use the variable we have formulated in 3.1 and 3.2 to calculate the average rain intensity for each city.

It calculates how much rain each city experiences per day on average.

In [86]:
# Merge the total_rainfall and rainy_days DataFrames on 'City'
rain_data = pd.merge(total_rainfall, rainy_days, on='City')

# Calculate the average rain intensity
rain_data['average_rain_intensity'] = rain_data['Total Rainfall'] / rain_data['Rainy Days']

# Rename the columns for clarity
rain_data.columns = ['City', 'Total Rainfall (mm)', 'Rainy Days', 'Average Rain Intensity (mm/day)']

# Display the result
display(rain_data)


,City,Total Rainfall (mm),Rainy Days,Average Rain Intensity (mm/day)
0,Buenos Aires,916.3,150,6.108667
1,Cairo,28.3,29,0.975862
2,London,780.7,228,3.424123
3,Mumbai,2048.1,157,13.045223
4,Singapore,2364.7,350,6.756286


Neat! Now we have 3/4 metrics represented in the table.

## 2.4 Average Rain Duration

We have to first calculate the total rain duration for each city. Then, calculate the average rain duration per rainy day for each city.

In [87]:
# Calculate Total Rain Duration for each city
total_rain_duration = df.groupby('city')['precipitation_hours'].sum().reset_index()
total_rain_duration.columns = ['City', 'Total Rain Duration']

# Merge the total_rain_duration with the rain_data DataFrame
rain_data = pd.merge(rain_data, total_rain_duration, on='City')

# Calculate Average Rain Duration per Rainy Day for each city
rain_data['Average Rain Duration (hr/day)'] = rain_data['Total Rain Duration'] / rain_data['Rainy Days']

# Display the result
display(rain_data[['City', 'Average Rain Duration (hr/day)']])

,City,Average Rain Duration (hr/day)
0,Buenos Aires,5.700000
1,Cairo,2.827586
2,London,6.741228
3,Mumbai,14.050955
4,Singapore,8.522857


## 2.5 Combining all metrics together

Now that we have all our relevant metrics, let's put them all together in one table to help our analysis.

In [88]:
# Merge the total_rainfall, rainy_days, and total_rain_duration DataFrames on 'City'
rain_data = pd.merge(total_rainfall, rainy_days, on='City')
rain_data = pd.merge(rain_data, total_rain_duration, on='City')

# Calculate the Average Rain Intensity and Average Rain Duration per Rainy Day for each city
rain_data['Average Rain Intensity'] = rain_data['Total Rainfall'] / rain_data['Rainy Days']
rain_data['Average Rain Duration'] = rain_data['Total Rain Duration'] / rain_data['Rainy Days']

# Display the final DataFrame with the specified columns
display(rain_data[['City', 'Total Rainfall', 'Rainy Days', 'Average Rain Intensity', 'Average Rain Duration']])


,City,Total Rainfall,Rainy Days,Average Rain Intensity,Average Rain Duration
0,Buenos Aires,916.3,150,6.108667,5.700000
1,Cairo,28.3,29,0.975862,2.827586
2,London,780.7,228,3.424123,6.741228
3,Mumbai,2048.1,157,13.045223,14.050955
4,Singapore,2364.7,350,6.756286,8.522857


# 3. Visualisations

In [89]:
from lets_plot import *
LetsPlot.setup_html()

## 3.1 Comparative Rain Metrics Across Cities (Heatmap)

This will allow us to compare the total rainfall for each city over the entire year.

In [127]:
# Customize the heatmap
p = (ggplot(rain_data_melted, aes(x='City', y='Metric', fill='Value'))
     + geom_tile(color='black', size=0.5)
     + geom_text(aes(label='Value'), color='black', size=10)  # Display values on tiles
     + scale_fill_gradient2(
         low="#D4E6F1", mid="#7FB3D5", high="#154360", midpoint=rain_data_melted['Value'].mean(),  # Divergent color scale
         guide='colorbar', limits=[rain_data_melted['Value'].min(), rain_data_melted['Value'].max()]
     )
     + ggtitle("Weather Metrics Comparison Across Major Cities (2023)")
     + theme_minimal()
     + labs(x="City", y="Metric", fill="Value")
     + theme(axis_text_x=element_text(angle=0, hjust=1, size=20),
             axis_text_y=element_text(size=20),
             plot_title=element_text(size=25, face='bold'),
             legend_title=element_text(size=30, face='bold'),
             legend_text=element_text(size=15),  # Larger legend labels
             legend_position='right')  # Customize legend position and title
     + ggsize(1000, 800)  # Larger plot size
)

# Display the plot
p.show()

Conclusion: London has moderate total rainfall compared to other cities, with significantly less than monsoon-prone cities like Mumbai and tropical cities like Singapore. This suggests that London’s overall rainfall volume isn’t exceptionally high.

## 3.2 Number of Rainy Days by City (Bar Chart)

This will allow us to compare the total rainfall for each city over the entire year.

In [113]:
# Prepare data for number of rainy days by city with method chaining
rainy_days_data = (
    rain_data[['City', 'Rainy Days']]
    .assign(City=lambda df: df['City'].astype('category'))
)

# Create the bar chart with advanced customization
plot = (ggplot(rainy_days_data, aes(x='City', y='Rainy Days', fill='Rainy Days')) + 
       geom_bar(stat='identity', show_legend=False) + 
       geom_text(aes(label='Rainy Days'), position=position_nudge(y=5.5), color='black', size=10) + 
       scale_fill_manual(values=['#92C5F9', '#4394E5', '#0066CC', '#004D99', '#003366']) + 
       ggtitle('Rain or shine? In 2023, Singapore was mostly rain.', subtitle='London experienced ~35% less rainy days in comparison to SG.') + 
       theme_minimal() + 
       theme(axis_title_x=element_text(size=20), 
             axis_title_y=element_text(size=20),
             plot_title=element_text(size=30, face="bold", color='#333333'),
             plot_subtitle=element_text(size=20, color="black"),  # Adjusted subtitle size
             axis_text_x=element_text(size=20),
             axis_text_y=element_text(size=20),
             panel_grid_major_y=element_line(color="grey", size=0.3, linetype="dotted"),
             panel_grid_major_x=element_blank(),
             legend_position='none') + 
       labs(x='City', y='Number of Rainy Days')+ ggsize(1200, 800))

# Display the plot
plot

## 3.3 Monthly Total Rainfall by City (Line Graph)

This line graph shows the monthly rainfall for each city, revealing seasonal trends and allowing comparison of rain patterns over time.

In [122]:
# Convert 'time' column to datetime
df['time'] = pd.to_datetime(df['time'])

# Extract month and year from 'time' column
df['month'] = df['time'].dt.to_period('M')

# Prepare data for monthly total rainfall by city with method chaining
monthly_rainfall_data = (
    df.groupby(['city', 'month'])['precipitation_sum']
    .sum()
    .reset_index()
    .assign(month=lambda df: df['month'].astype(str))
)

# Create the line graph with advanced customization
plot = (ggplot(monthly_rainfall_data, aes(x='month', y='precipitation_sum', color='city', group='city')) + \
       geom_line(size=1.5) + 
       geom_point(size=3) + 
       scale_color_manual(values=['#92C5F9', '#4394E5', '#0066CC', '#004D99', '#003366']) + 
       ggtitle('Rainfall Roller Coaster: Peaks and Patterns Across Cities', subtitle='Mumbai leads with a sharp monsoon spike while other cities show consistent trend') + \
       theme_minimal() + 
       theme(axis_title_x=element_text(size=20, face="bold"), 
             axis_title_y=element_text(size=20, face="bold"),
             plot_title=element_text(size=30, face="bold", color='#333333'),
             plot_subtitle=element_text(size=20, color="gray"),
             axis_text_x=element_text(size=20, hjust=1, face="bold"),
             axis_text_y=element_text(size=20, face="bold"),
             panel_grid_major_y=element_line(color="lightgray", size=0.5, linetype="dotted"),
             panel_grid_major_x=element_blank(),
             legend_position='right') + 
       scale_x_discrete(name='Month', labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']) + 
       labs(x='Month', y='Total Rainfall (mm)', color='City') + ggsize(1200, 1000))

# Display the plot
plot


Conclusion: London shows consistent rainfall throughout the year, unlike cities with distinct rainy seasons (e.g., Mumbai’s monsoon spike). This steady pattern can give the impression of constant rain, even if it’s relatively light.

## 3.5 

## 3.5 Average Rain Intensity by City (Bar Graph)

In [123]:
# Prepare data for average rain intensity by city with method chaining
average_rain_intensity_data = (
    rain_data[['City', 'Average Rain Intensity']]
    .assign(City=lambda df: df['City'].astype('category'))
 )

# Create the bar chart with advanced customization
plot = (ggplot(average_rain_intensity_data, aes(x='City', y='Average Rain Intensity', fill='City')) + 
       geom_bar(stat='identity', show_legend=False) + 
       geom_text(aes(label='Average Rain Intensity'), position=position_nudge(y=0.25), color='black', size=6.5) + 
       scale_fill_manual(values=['#92C5F9', '#4394E5', '#0066CC', '#004D99', '#003366']) + \
       ggtitle('Average Rain Intensity by City (2023)', subtitle='A comparative analysis of rain intensity across cities') + 
       theme_minimal() + 
       theme(axis_title_x=element_text(size=12), 
             axis_title_y=element_text(size=12),
             plot_title=element_text(size=18, face="bold", color='#333333'),
             plot_subtitle=element_text(size=14, color="gray"),
             axis_text_x=element_text(size=11, hjust=1),
             axis_text_y=element_text(size=11),
             panel_grid_major_y=element_line(color="lightgray", size=0.5, linetype="dotted"),
             panel_grid_major_x=element_blank(),
             legend_position='none') + 
       labs(x='City', y='Average Rain Intensity (mm/day)') + ggsize(1200, 500))

# Display the plot
plot

Conclusion: London’s rain intensity is lower than cities like Mumbai and Singapore. This means that while it rains often, the rainfall itself is less intense, contradicting the notion of London being heavily rainy.

## Overall Conclusion ##
While London has frequent, consistent rain that contributes to its "rainy" reputation, its rainfall is generally light and moderate in volume compared to other cities with intense, seasonal rain. London’s raininess is more about persistence and drizzle than heavy downpours, aligning partially with the “rainy” portrayal in movies, though it’s less intense than often dramatized.